In [19]:
%pip install datasets trl peft bitsandbytes wandb accelerate transformers

Note: you may need to restart the kernel to use updated packages.


In [20]:
%pip install unsloth
%pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Note: you may need to restart the kernel to use updated packages.
Found existing installation: unsloth 2024.9.post2
Uninstalling unsloth-2024.9.post2:
  Successfully uninstalled unsloth-2024.9.post2
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ishtpvet/unsloth_1547be0b78554e87aadf76bcc766ba0f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ishtpvet/unsloth_1547be0b78554e87aadf76bcc766ba0f
  Resolved https://github.com/unslothai/unsloth.git to commit fb77505f8429566f5d21d6ea5318c342e8a67991
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.9.post2-py3-none-any.whl size=154625 sha256=1477ee3da89c1efac7f9d4005ef6dd6e102e6217f47b43df9ac9af233c6f2bf4
  Stored in directory: /tmp/pip-ephem-wheel-cache-xqx95zsl/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d83

In [21]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [22]:
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

In [23]:
from hmac import new
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
base_model = "unsloth/llama-3-8b-Instruct-bnb-4bit"
new_model = "./output/llama-3-8b-Instruct-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit", # Choose ANY! eg mistralai/Mistral-7B-Instruct-v0.2
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla V100-SXM2-32GB. Max memory: 31.739 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [24]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

In [25]:
from datasets import load_dataset

dataset = load_dataset("Intel/orca_dpo_pairs")['train']

In [26]:
dataset = dataset.shuffle(seed=42).select(range(150))

In [27]:
dataset

Dataset({
    features: ['system', 'question', 'chosen', 'rejected'],
    num_rows: 150
})

In [28]:
dataset[47]['question']

'Article: People like to read stories about dog very much.They thought dogs were very clever. One of my good friend , Bob , had a very big dog .Its name was Bill.Every Sunday afternoon,Bob and Bill had a walk in the park , Bill liked walking with Bob very much. I visited Bob last Sunday.I stayed in his house for a long time.Bob and I talked with each other happily.Soon it was time for them to go for a walk in the park .We forgot that.Bill began to worry about it.He walked around the room and sat down in front of me and looked at me.But I didn\'t know this.I went on talking with my friend.At last, Bill couldn\'t wait.He went out of the room and came back soon.He carried my hat in his mouth.Oh, I knew what Bill meant. Question: How many people were there in this story? Yes or no, is the answer "Two"?\nA:'

In [29]:
dataset[47]['rejected']

' Sure, I\'d be happy to help! To answer your question, the story has three people:\n\n1. Bob (the owner of the dog)\n2. Bill (the dog)\n3. You (the person who visited Bob and is asking the question)\n\nSo, the answer to your question is "No" because there are not two people in the story. There are three people.'

In [30]:
dataset[47]['chosen']

'In this story, there were three people mentioned: the narrator, their good friend Bob, and the person visiting Bob on Sunday. So the answer to the question "How many people were there in this story?" is three. Therefore, "Two" is not the correct answer, so the response for "Yes or no, is the answer \'Two\'?" would be no.'

In [31]:
chat_template = open('llama-3-instruct.jinja').read()
chat_template

"{% if messages[0]['role'] == 'system' %}\n    {% set offset = 1 %}\n{% else %}\n    {% set offset = 0 %}\n{% endif %}\n\n{{ bos_token }}\n{% for message in messages %}\n    {% if (message['role'] == 'user') != (loop.index0 % 2 == offset) %}\n        {{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}\n    {% endif %}\n\n    {{ '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n' + message['content'] | trim + '<|eot_id|>' }}\n{% endfor %}\n\n{% if add_generation_prompt %}\n    {{ '<|start_header_id|>' + 'assistant' + '<|end_header_id|>\\n\\n' }}\n{% endif %}"

In [32]:
chat_template = chat_template.replace('    ', '').replace('\n', '')
tokenizer.chat_template = chat_template
chat_template

"{% if messages[0]['role'] == 'system' %}{% set offset = 1 %}{% else %}{% set offset = 0 %}{% endif %}{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == offset) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{{ '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n' + message['content'] | trim + '<|eot_id|>' }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>' + 'assistant' + '<|end_header_id|>\\n\\n' }}{% endif %}"

In [33]:
def dataset_format(example):
    # Format system
    if len(example["system"]) > 0:
        message = {"role": "system", "content": example["system"]}
        system = tokenizer.apply_chat_template([message], tokenize=False)
    else:
        system = ""
    # Format instruction
    message = {"role": "user", "content": example["question"]}
    prompt = tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)
    # Format chosen answer
    chosen = example["chosen"] + "<|eot_id|>\n"
    # Format rejected answer
    rejected = example["rejected"] + "<|eot_id|>\n"
    return {
        "prompt": system + prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

In [34]:
original_columns = dataset.column_names
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

dataset = dataset.map(
    dataset_format,
    remove_columns=original_columns,
    num_proc=os.cpu_count(),
)

Map (num_proc=96):   0%|          | 0/150 [00:00<?, ? examples/s]

In [35]:
import wandb

wandb.login()

True

In [36]:
from trl import DPOConfig, DPOTrainer
from unsloth import is_bfloat16_supported


dpo_config = DPOConfig(
    output_dir=new_model,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=50,  # tweak this to change # of steps in the training run
    save_strategy="no",
    logging_steps=1,
    optim="paged_adamw_32bit",
    warmup_steps=10,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    report_to="wandb",
    beta=0.1,
    max_prompt_length=512,
    max_length=1024,
    force_use_ref_model=True,
    remove_unused_columns=False,
)

In [37]:
dpo_trainer = DPOTrainer(
    model,
    None,
    args=dpo_config,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

Tokenizing train dataset:   0%|          | 0/150 [00:00<?, ? examples/s]

Detected kernel version 4.9.70, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs
